In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier,RandomForestRegressor,BaggingRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression,Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
import sklearn.metrics as metrics
from sklearn.metrics import r2_score,roc_auc_score,classification_report,mean_squared_error,accuracy_score,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [24]:
df=pd.read_csv('D:/GLH/train-new.csv')

In [25]:
df1=df.copy()

In [27]:
df1.shape

(17565, 15)

In [26]:
df1.isnull().sum()

dvcat          0
weight         0
dead           0
airbag         0
seatbelt       0
frontal        0
sex            0
ageOFocc       0
yearacc        0
yearVeh        0
abcat          0
occRole        0
deploy         0
caseid         0
injSeverity    0
dtype: int64

In [28]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17565 entries, 0 to 17564
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dvcat        17565 non-null  object 
 1   weight       17565 non-null  float64
 2   dead         17565 non-null  object 
 3   airbag       17565 non-null  object 
 4   seatbelt     17565 non-null  object 
 5   frontal      17565 non-null  int64  
 6   sex          17565 non-null  object 
 7   ageOFocc     17565 non-null  int64  
 8   yearacc      17565 non-null  int64  
 9   yearVeh      17565 non-null  int64  
 10  abcat        17565 non-null  object 
 11  occRole      17565 non-null  object 
 12  deploy       17565 non-null  int64  
 13  caseid       17565 non-null  object 
 14  injSeverity  17565 non-null  int64  
dtypes: float64(1), int64(6), object(8)
memory usage: 2.0+ MB


In [30]:
df1.head()

,dvcat,weight,dead,airbag,seatbelt,frontal,sex,ageOFocc,yearacc,yearVeh,abcat,occRole,deploy,caseid,injSeverity
0,55+,53.342,dead,airbag,belted,1,f,48,2002,1997,deploy,driver,1,45:150:1,3
1,25-39,154.960,alive,none,none,1,m,26,2001,1968,unavail,driver,0,76:40:1,3
2,55+,38.994,alive,none,none,1,f,51,2002,1994,unavail,driver,0,11:184:1,3
3,25-39,168.568,alive,airbag,belted,1,m,27,1998,1996,deploy,pass,1,9:17:1,3
4,10-24,27.751,alive,airbag,belted,0,m,26,2002,1997,nodeploy,pass,0,2:2:2,0


In [31]:
df1['dvcat'].unique()

array(['55+', '25-39', '10-24', '40-54', '1-9km/h'], dtype=object)

In [43]:
# convert impacted speeds.
# df1['dvcat'].replace({'1-9km/h':'low impact','10-24':'medium impact','25-39':'high impact',
#                       '40-54':'high impact',
#                      '55+':'critical impact'})
df1['dvcat'].replace({'1-9km/h':1,'10-24':2,'25-39':3,
                      '40-54':4,
                     '55+':5},inplace=True)

In [52]:
# weight is already in continuous numeric,so we wont touch it.
# frontal is already in encoded format,so we wont touch it.
# 'caseid' 1st drop it,then try keeping it

In [60]:
# only 2 catgories in dead feature,so we can label encode or manually encode:
df1['dead'].replace({'dead':0,'alive':1},inplace=True)

In [64]:
# only 2 categories in airbag feature,again,so we can label encode or manually encode:
df1['airbag'].replace({'none':0,'airbag':1},inplace=True)

In [66]:
# only 2 categories in seatbelt feature,again,so we can label encode or manually encode:
df1['seatbelt'].replace({'none':0,'belted':1},inplace=True)

In [69]:
# only 2 categories in sex feature,again,so we can label encode or manually encode:
df1['sex'].replace({'f':0,'m':1},inplace=True)

In [73]:
# we will try with age category as it is for the 1st time,check accuracy then see if it makes any difference 
# by grouping people agewise

In [76]:
# year has nothing to do with how accidents happen,so,we will drop it in 1st part.
# later keeping it we will check


# year of vehicle,we will keep it,this is because if the car model is very old and is not maintained 
# properly,there is strong possibilty that it will crash,so we will keep it as it is
# 2nd time we will try with grouping them car age wise and compare it with year it crashed,so we will
# get an idea when exactly did car crash,so was it because of car was old and not maintained properly?

In [79]:
# 3 classes,airbags deployed or not / unavailable
df1['abcat'].replace({'unavail':0,'nodeploy':1,'deploy':2},inplace=True)

In [82]:
# only 2 categories in sex feature,again,so we can label encode or manually encode:
# we will consider driver as driver with no pass,so 0 and pass as 1
df1['occRole'].replace({'driver':0,'pass':1},inplace=True)

In [88]:
df1.drop('caseid',axis=1,inplace=True)

df1.drop('yearacc',axis=1,inplace=True)

### Splitting the data:

In [93]:
X=df1.drop('injSeverity',1)
y=df1['injSeverity']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =0)

In [107]:
X=df1.drop('injSeverity',1)
y=df1['injSeverity']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =0)
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
dt = DecisionTreeClassifier()
dt_params = {'max_depth':np.arange(1,10), 'min_samples_leaf':np.arange(2,30), 'criterion':['entropy','gini']}
rscv = RandomizedSearchCV(dt, dt_params, cv=5)
rscv.fit(X, y)
print(rscv.best_params_)
rscv_best_DT=rscv.best_params_
DT=DecisionTreeClassifier(**rscv_best_DT)
DT.fit(X_train,y_train)
y_pred=DT.predict(X_test)
y_pred_proba=DT.predict_proba(X_test)[:,1]
#{'min_samples_leaf': 21, 'max_depth': 6, 'criterion': 'gini'}

{'min_samples_leaf': 21, 'max_depth': 6, 'criterion': 'gini'}


In [131]:
# pd.Series(y_pred,name='ypred')

#### Test data manipulations:

In [173]:
dfte=pd.read_csv('D:/GLH/test-new.csv')
df2=dfte.copy()
# df2.dropna(inplace=True)

In [177]:
df2['yearVeh'].fillna(1995,inplace=True) # most frequent value

In [174]:
df2['dvcat'].replace({'1-9km/h':1,'10-24':2,'25-39':3,
                      '40-54':4,
                     '55+':5},inplace=True)
df2['dead'].replace({'dead':0,'alive':1},inplace=True)
df2['airbag'].replace({'none':0,'airbag':1},inplace=True)
df2['seatbelt'].replace({'none':0,'belted':1},inplace=True)
df2['sex'].replace({'f':0,'m':1},inplace=True)
df2['abcat'].replace({'unavail':0,'nodeploy':1,'deploy':2},inplace=True)
df2['occRole'].replace({'driver':0,'pass':1},inplace=True)
df2.drop('caseid',axis=1,inplace=True)
df2.drop('yearacc',axis=1,inplace=True)

In [178]:
df2.shape

(8652, 12)

## Logistic Regression:

In [179]:
lr=LogisticRegression()
lr.fit(X,y)
y_test_pred_lr=lr.predict(df2)
finalpred=pd.DataFrame(y_test_pred_lr,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\lrpred.csv",index=False)

## DT classifier:

In [180]:
X=df1.drop('injSeverity',1)
y=df1['injSeverity']
DT.fit(X,y)
y_test_pred_lr=DT.predict(df2)
finalpred=pd.DataFrame(y_test_pred_lr,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\dtpred.csv",index=False)

# RF classifier:

In [183]:
RF=RandomForestClassifier()
RF.fit(X,y)
y_test_pred_RF=DT.predict(df2)
finalpred=pd.DataFrame(y_test_pred_RF,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\RFpred.csv",index=False)

# DT With param tuning [grid search]:

In [185]:
X=df1.drop('injSeverity',1)
y=df1['injSeverity']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =0)
    from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
    dt = DecisionTreeClassifier()
    dt_params = {'max_depth':np.arange(1,10), 'min_samples_leaf':np.arange(2,30), 'criterion':['entropy','gini']}
    rscv = GridSearchCV(dt, dt_params, cv=5)
    rscv.fit(X, y)
    print(rscv.best_params_)
rscv_best_DT=rscv.best_params_
DT=DecisionTreeClassifier(**rscv_best_DT)

X=df1.drop('injSeverity',1)
y=df1['injSeverity']
DT=DecisionTreeClassifier(**rscv_best_DT)
DT.fit(X,y)
y_test_pred_dt=DT.predict(df2)
finalpred=pd.DataFrame(y_test_pred_dt,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\dtpredparams.csv",index=False)
#{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 7} *50.56 score

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 7}


# RF Tuned[Randomized]:

In [189]:
X=df1.drop('injSeverity',1)
y=df1['injSeverity']
RF=RandomForestClassifier()
params={
 'bootstrap': [True, False],
 'max_depth': [10, 20, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [10,30,50,100,125,150]
}
rscv = RandomizedSearchCV(RF, param_distributions=params, cv=5)
rscv.fit(X, y)
print(rscv.best_params_)
rscv_best_RF=rscv.best_params_

{'n_estimators': 125, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False}


In [191]:
RF=RandomForestClassifier(**rscv_best_RF)
RF.fit(X,y)
y_test_pred_RF=RF.predict(df2)
finalpred=pd.DataFrame(y_test_pred_RF,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\RFpred_paramtuned.csv",index=False)
# 51.53 score on leaderboard with random param-
#{'n_estimators': 125, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False}

# XGB classifier:

In [200]:
XGB=XGBClassifier()
XGB.fit(X,y)
y_test_pred_XGB=XGB.predict(df2)
finalpred=pd.DataFrame(y_test_pred_XGB,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\XGB.csv",index=False)

## 51.61 best score so far.

# Randomized XGB:

In [202]:
X=df1.drop('injSeverity',1)
y=df1['injSeverity']
XGB=XGBClassifier()
params={
 'bootstrap': [True, False],
 'max_depth': [10, 20, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [10,30,50,100,125,150]
}
rscv = RandomizedSearchCV(XGB, param_distributions=params, cv=5)
rscv.fit(X, y)
print(rscv.best_params_)
rscv_best_XGB=rscv.best_params_

##{'n_estimators': 50, 'min_samples_split': 2,'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': True}

{'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': True}


In [203]:
XGB=XGBClassifier(**rscv_best_XGB)
XGB.fit(X,y)
y_test_pred_XGB=XGB.predict(df2)
finalpred=pd.DataFrame(y_test_pred_XGB,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\XGBrandomized.csv",index=False)
# This is not giving too good results -50.43

# Train 1st data modification:

In [244]:
df3=df.copy()

In [245]:
df3['dvcat'].replace({'1-9km/h':1,'10-24':2,'25-39':3,
                      '40-54':4,
                     '55+':5},inplace=True)
df3['dead'].replace({'dead':0,'alive':1},inplace=True)
df3['airbag'].replace({'none':0,'airbag':1},inplace=True)
df3['seatbelt'].replace({'none':0,'belted':1},inplace=True)
df3['sex'].replace({'f':0,'m':1},inplace=True)
df3['abcat'].replace({'unavail':0,'nodeploy':1,'deploy':2},inplace=True)
df3['occRole'].replace({'driver':0,'pass':1},inplace=True)
df3.drop('caseid',axis=1,inplace=True)
# df3.drop('yearacc',axis=1,inplace=True)

#### Feature creation yeargap:

In [246]:
df3['yeargap']=df3['yearacc']-df3['yearVeh']

In [247]:
df3.corr()[['yeargap','yearVeh','yearacc']].iloc[-1:,:]
# High correlation with yearVeh

,yeargap,yearVeh,yearacc
yeargap,1.0,-0.952145,0.008692


In [248]:
df3.drop('yearVeh',axis=1,inplace=True)

## Test 1st data modifications:

In [250]:
dfte=pd.read_csv('D:/GLH/test-new.csv')
df4=dfte.copy()
# df4.dropna(inplace=True)
df4['dvcat'].replace({'1-9km/h':1,'10-24':2,'25-39':3,
                      '40-54':4,
                     '55+':5},inplace=True)
df4['dead'].replace({'dead':0,'alive':1},inplace=True)
df4['airbag'].replace({'none':0,'airbag':1},inplace=True)
df4['seatbelt'].replace({'none':0,'belted':1},inplace=True)
df4['sex'].replace({'f':0,'m':1},inplace=True)
df4['abcat'].replace({'unavail':0,'nodeploy':1,'deploy':2},inplace=True)
df4['occRole'].replace({'driver':0,'pass':1},inplace=True)
df4.drop('caseid',axis=1,inplace=True)
# df4.drop('yearacc',axis=1,inplace=True)
df4['yeargap']=df4['yearacc']-df4['yearVeh']
df4.drop('yearVeh',axis=1,inplace=True)

In [251]:
df4.fillna(1,inplace=True)

#### Logistic Regression:

In [256]:
X=df3.drop('injSeverity',1)
y=df3['injSeverity']

In [262]:
lr=LogisticRegression()
lr.fit(X,y)
y_test_pred_lr=lr.predict(df4)
finalpred=pd.DataFrame(y_test_pred_lr,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\lrpredmodif.csv",index=False)
# worst performance 42.53

#### Decision Tree:

In [264]:
X=df3.drop('injSeverity',1)
y=df3['injSeverity']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =0)
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
dt = DecisionTreeClassifier()
dt_params = {'max_depth':np.arange(1,10), 'min_samples_leaf':np.arange(2,30), 'criterion':['entropy','gini']}
rscv = GridSearchCV(dt, dt_params, cv=5)
rscv.fit(X, y)
print(rscv.best_params_)
rscv_best_DT=rscv.best_params_
DT=DecisionTreeClassifier(**rscv_best_DT)
DT.fit(X,y)
y_test_pred_dt=DT.predict(df4)
finalpred=pd.DataFrame(y_test_pred_dt,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\dtmodifiedrandomized.csv",index=False)

## 50.56 on wecp using  {'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 8}

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 8}


In [266]:
X=df3.drop('injSeverity',1)
y=df3['injSeverity']
RF=RandomForestClassifier()
params={
 'bootstrap': [True, False],
 'max_depth': [10, 20, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [10,30,50,100,125,150]
}
rscv = RandomizedSearchCV(RF, param_distributions=params, cv=5)
rscv.fit(X, y)
print(rscv.best_params_)
rscv_best_RF=rscv.best_params_
# {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False}

{'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False}


In [267]:
RF=RandomForestClassifier(**rscv_best_RF)
RF.fit(X,y)
y_test_pred_RF=RF.predict(df4)
finalpred=pd.DataFrame(y_test_pred_RF,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\modifieddataRf.csv",index=False)
#score on leaderboard with random param-51.58
#{'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False}

#### XGB classifier:[best combination]: 

In [270]:
XGB=XGBClassifier()
XGB.fit(X,y)
y_test_pred_XGB=XGB.predict(df4)
finalpred=pd.DataFrame(y_test_pred_XGB,columns=['injSeverity'])
finalpred.to_csv("D:/GLH/predictions/afterdatamodficationXGB.csv",index=False)
## 51.66 best one so far!

#### XGB with randomized data modified:

In [273]:
X=df3.drop('injSeverity',1)
y=df3['injSeverity']
XGB=XGBClassifier()
params={
 'bootstrap': [True, False],
 'max_depth': [10, 20],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 7],
 'n_estimators': [10,30,50]
}
rscv = RandomizedSearchCV(XGB, param_distributions=params, cv=5)
rscv.fit(X, y)
print(rscv.best_params_)
rscv_best_XGB=rscv.best_params_

#{'n_estimators': 50, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}

{'n_estimators': 50, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}


In [274]:
XGB=XGBClassifier(**rscv_best_XGB)
XGB.fit(X,y)
y_test_pred_XGB=XGB.predict(df4)
finalpred=pd.DataFrame(y_test_pred_XGB,columns=['injSeverity'])
finalpred.to_csv("D:/GLH/predictions/randomizedafterdatamodficationXGB.csv",index=False)
# 50.82,not so good results!

# 2nd data modification train:[combo of LE + get_dummies]

In [271]:
df5=df.copy()

In [275]:
# df5['dvcat'].replace({'1-9km/h':1,'10-24':2,'25-39':3,
#                       '40-54':4,
#                      '55+':5},inplace=True)
df5['dead'].replace({'dead':0,'alive':1},inplace=True)
df5['airbag'].replace({'none':0,'airbag':1},inplace=True)
df5['seatbelt'].replace({'none':0,'belted':1},inplace=True)
df5['sex'].replace({'f':0,'m':1},inplace=True)
# df5['abcat'].replace({'unavail':0,'nodeploy':1,'deploy':2},inplace=True)
df5['occRole'].replace({'driver':0,'pass':1},inplace=True)
df5.drop('caseid',axis=1,inplace=True)
df5['yeargap']=df5['yearacc']-df5['yearVeh']
df5.drop('yearVeh',axis=1,inplace=True)

In [278]:
df5=pd.get_dummies(df5,drop_first=True)

# 2nd data modification for test file:[combo LE + get_dummies]

In [284]:
dfte=pd.read_csv('D:/GLH/test-new.csv')
df6=dfte.copy()
# df6['dvcat'].replace({'1-9km/h':1,'10-24':2,'25-39':3,
#                       '40-54':4,
#                      '55+':5},inplace=True)
df6['dead'].replace({'dead':0,'alive':1},inplace=True)
df6['airbag'].replace({'none':0,'airbag':1},inplace=True)
df6['seatbelt'].replace({'none':0,'belted':1},inplace=True)
df6['sex'].replace({'f':0,'m':1},inplace=True)
# df6['abcat'].replace({'unavail':0,'nodeploy':1,'deploy':2},inplace=True)
df6['occRole'].replace({'driver':0,'pass':1},inplace=True)
df6.drop('caseid',axis=1,inplace=True)
# df6.drop('yearacc',axis=1,inplace=True)
df6['yeargap']=df6['yearacc']-df6['yearVeh']
df6.drop('yearVeh',axis=1,inplace=True)
df6.fillna(1,inplace=True)

In [289]:
df6=pd.get_dummies(df6,drop_first=True)

In [290]:
X=df5.drop('injSeverity',1)
y=df5['injSeverity']

#### Logistic regression:

In [291]:
lr=LogisticRegression()
lr.fit(X,y)
y_test_pred_lr=lr.predict(df6)
finalpred=pd.DataFrame(y_test_pred_lr,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\lr2ndmodified.csv",index=False)
# 42.28

#### Decision tree:

In [292]:
X=df5.drop('injSeverity',1)
y=df5['injSeverity']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =0)
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
dt = DecisionTreeClassifier()
dt_params = {'max_depth':np.arange(1,10), 'min_samples_leaf':np.arange(2,30), 'criterion':['entropy','gini']}
rscv = GridSearchCV(dt, dt_params, cv=5)
rscv.fit(X, y)
print(rscv.best_params_)
rscv_best_DT=rscv.best_params_
DT=DecisionTreeClassifier(**rscv_best_DT)
DT.fit(X,y)
y_test_pred_dt=DT.predict(df6)
finalpred=pd.DataFrame(y_test_pred_dt,columns=['injSeverity'])
finalpred.to_csv("D:\GLH\predictions\DT2ndgridsearch.csv",index=False)
#50.20

{'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 11}


#### Random Forest:

In [293]:
X=df5.drop('injSeverity',1)
y=df5['injSeverity']
RF=RandomForestClassifier()
params={
 'bootstrap': [True, False],
 'max_depth': [10, 20, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [10,30,50,100,125]
}
rscv = RandomizedSearchCV(RF, param_distributions=params, cv=5)
rscv.fit(X, y)
print(rscv.best_params_)
rscv_best_RF=rscv.best_params_

{'n_estimators': 125, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False}


In [295]:
RF=RandomForestClassifier(**rscv_best_RF)
RF.fit(X,y)
y_test_pred_RF=RF.predict(df6)
finalpred=pd.DataFrame(y_test_pred_RF,columns=['injSeverity'])
finalpred.to_csv("D:/GLH/predictions/2ndmodifieddatarandomizedrf.csv",index=False)
#51.53

#### XGB without params:

In [296]:
XGB=XGBClassifier()
XGB.fit(X,y)
y_test_pred_XGB=XGB.predict(df6)
finalpred=pd.DataFrame(y_test_pred_XGB,columns=['injSeverity'])
finalpred.to_csv("D:/GLH/predictions/2nddatamodficationXGB.csv",index=False)
## 51.25 best one so far!

### XGB with params:[ for 1st data] best than the rest:

In [305]:
X=df1.drop('injSeverity',1)
y=df1['injSeverity']

In [308]:
# params = {"objective": "multi:softmax","booster": "gbtree", "nthread": 4, "silent": 1,
#                 "eta": 0.08, "max_depth": 6, "subsample": 0.9, "colsample_bytree": 0.7,
#                 "min_child_weight": 1, "num_class": 3,
#                 "seed": 2016, "tree_method": "exact"}
# Randomized param output

In [307]:
XGB=XGBClassifier(**params)
XGB.fit(X,y)
y_test_pred_XGB=XGB.predict(df2)
finalpred=pd.DataFrame(y_test_pred_XGB,columns=['injSeverity'])
finalpred.to_csv("D:/GLH/predictions/2nddatamodficationXGB3rd.csv",index=False)
## 51.99 score 

## Ignore b/m part

In [310]:
XGB=XGBClassifier(learning_rate=0.09,n_estimators=125,max_depth=4,min_child_weight=4,colsample_bytree=0.5,reg_alpha=0.000001 )
XGB.fit(X,y)
y_test_pred_XGB=XGB.predict(df2)
finalpred=pd.DataFrame(y_test_pred_XGB,columns=['injSeverity'])
finalpred.to_csv("D:/GLH/predictions/1stdiffparamXGB.csv",index=False)
# 51.76

In [314]:
X=df3.drop('injSeverity',1)
y=df3['injSeverity']
XGB=XGBClassifier(learning_rate=0.09,n_estimators=125,max_depth=4,min_child_weight=4,colsample_bytree=0.5,reg_alpha=0.000001)
XGB.fit(X,y)
y_test_pred_XGB=XGB.predict(df4)
finalpred=pd.DataFrame(y_test_pred_XGB,columns=['injSeverity'])
finalpred.to_csv("D:/GLH/predictions/2nddiffparamXGB.csv",index=False)
#51.86

In [315]:
X=df5.drop('injSeverity',1)
y=df5['injSeverity']
XGB=XGBClassifier(learning_rate=0.09,n_estimators=125,max_depth=4,min_child_weight=4,colsample_bytree=0.5,reg_alpha=0.000001)
XGB.fit(X,y)
y_test_pred_XGB=XGB.predict(df6)
finalpred=pd.DataFrame(y_test_pred_XGB,columns=['injSeverity'])
finalpred.to_csv("D:/GLH/predictions/3rddiffparamXGB.csv",index=False)
#51.43